In [2]:
from getData import df
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [9]:

import pandas as pd
from parsedData import parsedData as data

df = pd.DataFrame(data, columns=['Date', 'Time', 'Author', 'Message', 'Conversation'])

authors = df['Author'].unique()
conversations = df['Conversation'].unique()

df = df.drop(df[df['Author'].isnull()].index)
df['Word Count'] = df['Message'].apply(lambda s : len(s.split(' ')))

df['Datetime'] = df['Date'] +' '+ df['Time']
df['Datetime'] = pd.to_datetime(df['Datetime'])

df['Sent Messages'] = 1
df['Sent Words'] = df['Word Count']
df['WPM'] = 0
df['Total Messages'] = 1
df['Total Words'] = df['Word Count']

new_df = pd.DataFrame()

for group in conversations:
    subset_group = df[df.Conversation == group].copy()
    authors = list(subset_group.Author.unique())
    
    subset_group['Total Messages'] = subset_group['Total Messages'].cumsum()
    subset_group['Total Words'] = subset_group['Total Words'].cumsum()

    for key in authors:
        subset = subset_group[subset_group.Author == key].copy()
        
        subset['Sent Messages'] = subset['Sent Messages'].cumsum()
        subset['Sent Words'] = subset['Sent Words'].cumsum()
        
        new_df = new_df.append(subset)
        

df = new_df[['Datetime', 'Author', 'Conversation', 'Sent Messages', 'Sent Words', 'WPM', 'Total Messages', 'Total Words']].copy()
df['WPM'] = df['Sent Words'] / df['Sent Messages']
df = df.sort_values(['Datetime'])




In [17]:
df[df.Conversation == df.Conversation.unique()[3]].tail()

,Datetime,Author,Conversation,Sent Messages,Sent Words,WPM,Total Messages,Total Words
807,2020-02-24 12:23:00,Miles Keating,Poop,383,1494,3.900783,808,3364
810,2020-02-24 12:23:00,Poop,Poop,427,1876,4.393443,811,3374
808,2020-02-24 12:23:00,Poop,Poop,426,1871,4.392019,809,3365
811,2020-02-24 13:55:00,Miles Keating,Poop,385,1499,3.893506,812,3375
812,2020-02-24 13:55:00,Miles Keating,Poop,386,1500,3.886010,813,3376
